In [207]:
import pandas as pd
import pandahouse as ph

from datetime import datetime

#import seaborn as sns
#import matplotlib.pyplot as plt
import numpy as np
#import re
#%matplotlib inline

In [208]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
'database':'default',
'user':'student',
'password':'dpo_python_2020'
}

In [209]:
# для данной таблицы запрос выдает ошибку через pandahouse при чтении таблицы logistics_online
# поэтому к ней я обращаюсь на прямую через файл
logistic_online = pd.read_csv('logistic_online.csv')

q2 = '''
SELECT
    *
FROM
    default.product_online
'''
product_online = ph.read_clickhouse(query=q2, connection=connection_default)

q3 = '''
SELECT
    *
FROM
    default.returns_online
'''
returns_online = ph.read_clickhouse(query=q3, connection=connection_default)


# Предобработка данных

# logistic_online

In [210]:
df_log = logistic_online

**date_order** - дата, в которую клиент оформляет заказ

**number_order** - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. заказы переносятся и имеют раные даты доставки)

**timeslot_number** - нумерация кол-ва попыток доставить в плановую дату доставки. Если доставка не была осуществлена, то заказ переносится на новую дату

**result_data_order** - дата, в которую заказ был доставлен клиенту

**delivery_type** - выбранный клиентом способ получения заказа (курьерка или самовывоз)

**carrier_name** - перевозчик, которым заказ доставляется клиенту

**seller_ID** - уникальный идентификатор продавца

**order_status** - последний актуальный статус заказа (фиксируется во всех строках последний статус заказа)

**customer_cluster** - региональная принадлежность клиента

**warehouse_cluster** - региональная принадлежность продавца

                                                Смотрим датафрейм

In [211]:
df_log.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-5-01,313655114,2021-14-01,1,2021-11-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-3-01,315229153,2021-11-01,1,2021-8-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-8-01,3114106095,2021-16-01,1,2021-14-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-12-01,31145541407,2021-20-01,1,2021-15-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-16-01,325050136,2021-26-01,1,2021-25-01,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [212]:
df_log.dtypes

date_order            object
number_order           int64
planned_data_order    object
timeslot_number        int64
result_data_order     object
delivery_type         object
carrier_name          object
seller_ID              int64
order_status          object
customer_cluster      object
warehouse_cluster     object
dtype: object

In [213]:
df_log.isna().sum()

date_order            625
number_order            0
planned_data_order      0
timeslot_number         0
result_data_order     890
delivery_type           0
carrier_name            0
seller_ID               0
order_status            0
customer_cluster        0
warehouse_cluster       0
dtype: int64

In [214]:
df_log.customer_cluster.value_counts()

Кластер Москва                1808
Кластер Екатеринбург          1133
Кластер Московская область     843
Кластер Краснодар              710
Кластер Санкт-Петербург        565
Кластер Казань                 512
Кластер Центр                  472
Кластер Новосибирск            225
Кластер Северо-Запад           200
Кластер Хабаровск               58
Кластер Калининград             37
Кластер Якутия                  34
Name: customer_cluster, dtype: int64

In [215]:
df_log.warehouse_cluster.value_counts()

Кластер Москва                1752
Кластер Екатеринбург          1148
Кластер Московская область    1075
Кластер Казань                 617
Кластер Краснодар              588
Кластер Санкт-Петербург        470
Кластер Новосибирск            390
Кластер Центр                  246
Кластер Северо-Запад           205
Кластер Калининград             96
Кластер Хабаровск               10
Name: warehouse_cluster, dtype: int64

In [216]:
df_log.delivery_type.value_counts()

Курьер       3502
Самовывоз    3095
Name: delivery_type, dtype: int64

In [217]:
df_log.carrier_name.value_counts()

Самодоставка    2109
3PL 2           1890
3PL 1           1827
3PL 3            517
3PL 4            254
Name: carrier_name, dtype: int64

In [218]:
df_log.order_status.value_counts()

Доставлен            5694
На пути к клиенту     423
Отменен               334
Ожидает сборки        142
Спор                    4
Name: order_status, dtype: int64

                                                    ВЫВОД
Необходимо привести даты к типу datetime в 3 колонках.

In [219]:
df_log = df_log.reset_index()

In [220]:
def correct_date(df, column):
    date_list1 = list()
    for i in df[column]:
        date = i.split('-')
            # исправляем день месяца (добавляем 0 в начало)
        if int(date[1]) < 10 and len(date[1]) > 2:
            date[1] = '0' + date[1]
            # исправляем номер месяца (оставляем 2 цифры)
        if len(date[2]) == 3:
            date[2] = date[2][1:]
        date1 = pd.to_datetime(date[0] + '-' + date[2] + '-' + date[1])
        date_list1.append(date1)
    return date_list1


In [221]:
#Датафрейм для корректировки date_order
df_date_order = df_log[['index','date_order']].dropna()

#Датафрейм для корректировки дат date_order 
df_date_order['correct_date_order'] = correct_date(df_date_order, 'date_order')
df_date_order = df_date_order.drop(columns=['date_order'])


In [222]:
#Датафрейм для корректировки date_order
df_planned_data_order = df_log[['index','planned_data_order']].dropna()

#Датафрейм для корректировки дат date_order 
df_planned_data_order['correct_planned_data_order'] = correct_date(df_planned_data_order, 'planned_data_order')
df_planned_data_order = df_planned_data_order.drop(columns=['planned_data_order'])

In [223]:
#Датафрейм для корректировки date_order
df_result_data_order = df_log[['index','result_data_order']].dropna()

#Датафрейм для корректировки дат date_order 
df_result_data_order['correct_result_data_order'] = correct_date(df_result_data_order, 'result_data_order')
df_result_data_order = df_result_data_order.drop(columns=['result_data_order'])


In [224]:
# Добавляем колонки с исправленными датами к исходному датафрейму
df_log = df_log.merge(df_date_order, on='index', how='left')
df_log = df_log.merge(df_planned_data_order, on='index', how='left')
df_log = df_log.merge(df_result_data_order, on='index', how='left')

In [225]:
df_log.head()

,index,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,correct_date_order,correct_planned_data_order,correct_result_data_order
0,0,2021-5-01,313655114,2021-14-01,1,2021-11-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва,2021-01-05,2021-01-14,2021-01-11
1,1,2021-3-01,315229153,2021-11-01,1,2021-8-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,2021-01-03,2021-01-11,2021-01-08
2,2,2021-8-01,3114106095,2021-16-01,1,2021-14-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,2021-01-08,2021-01-16,2021-01-14
3,3,2021-12-01,31145541407,2021-20-01,1,2021-15-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва,2021-01-12,2021-01-20,2021-01-15
4,4,2021-16-01,325050136,2021-26-01,1,2021-25-01,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург,2021-01-16,2021-01-26,2021-01-25


In [226]:
# Приводим датафрейм к исходному виду с исправленными датами
df_log = df_log.drop(columns=['index', 'date_order', 'planned_data_order', 'result_data_order'])

df_log = df_log[['correct_date_order', 'number_order', 'correct_planned_data_order',
                'timeslot_number', 'correct_result_data_order', 'delivery_type', 'carrier_name',
                'seller_ID', 'order_status', 'customer_cluster', 'warehouse_cluster']]

df_log = df_log.rename(columns={'correct_date_order'        : 'date_order', 
                                'correct_planned_data_order': 'planned_data_order', 
                                'correct_result_data_order' : 'result_data_order'})

In [227]:
df_log.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [228]:
df_log.dtypes

date_order            datetime64[ns]
number_order                   int64
planned_data_order    datetime64[ns]
timeslot_number                int64
result_data_order     datetime64[ns]
delivery_type                 object
carrier_name                  object
seller_ID                      int64
order_status                  object
customer_cluster              object
warehouse_cluster             object
dtype: object

## После обработки посмотрим данные на корректность

- Проверим есть ли заказы, в которых result_data_order меньше, чем date_order (доставка завершена раньше самого заказа)

Такой ситуации не может быть, т. е. ошибка в данных и при подсчете метрики, эти заказы нам нужно обработать

In [229]:
df_log.query("date_order > result_data_order")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
5,2021-02-02,326224027,2021-02-06,1,2021-01-26,Курьер,Самодоставка,902912,Доставлен,Кластер Екатеринбург,Кластер Екатеринбург
6,2021-02-02,3264191401,2021-02-08,1,2021-01-28,Самовывоз,3PL 2,923125,Доставлен,Кластер Казань,Кластер Новосибирск
25,2021-03-08,347979023,2021-03-15,1,2021-03-04,Самовывоз,3PL 2,833469,Доставлен,Кластер Центр,Кластер Екатеринбург
30,2021-03-04,3511447022,2021-03-12,1,2021-03-03,Курьер,Самодоставка,902912,Доставлен,Кластер Екатеринбург,Кластер Екатеринбург
51,2021-01-12,3131450699,2021-01-18,1,2021-01-10,Самовывоз,3PL 2,833469,Доставлен,Кластер Казань,Кластер Екатеринбург
...,...,...,...,...,...,...,...,...,...,...,...
6557,2021-01-30,327535272,2021-02-08,1,2021-01-28,Курьер,Самодоставка,902912,Доставлен,Кластер Екатеринбург,Кластер Екатеринбург
6561,2021-02-19,33314414321,2021-03-01,1,2021-02-12,Самовывоз,3PL 2,833469,Доставлен,Кластер Центр,Кластер Екатеринбург
6574,2021-03-05,350310006,2021-03-10,1,2021-02-25,Курьер,Самодоставка,902912,Доставлен,Кластер Екатеринбург,Кластер Екатеринбург
6575,2021-03-04,350424617,2021-03-10,1,2021-02-25,Курьер,Самодоставка,902912,Доставлен,Кластер Екатеринбург,Кластер Екатеринбург


In [230]:
df_log.query("date_order > result_data_order")\
    .query("date_order > planned_data_order")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster


In [231]:
# Поменяем местами в данных заказах значения колонок result_data_order и date_order
    # Не совсем правильно так изменять данные, не разбираясь в природе ошибки
    # но у нас нет информации, чтобы решить куда определить их или как их скоректировать лучше
    # а терять данные о 15% заказов может быть опасно при принятии решения

df_log = df_log.reset_index()
df_log_c = df_log.query("date_order > result_data_order")\
    .rename(columns={'date_order': 'result_data_order', 'result_data_order': 'date_order'})
df_log = df_log.query("(date_order <= result_data_order) or date_order.isnull() or result_data_order.isnull()")
df_log = df_log.append(df_log_c).sort_values('index')
df_log = df_log.drop(columns='index')


In [232]:
df_log

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...
6592,2021-03-14,361544120,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,64865,Доставлен,Кластер Краснодар,Кластер Северо-Запад
6593,2021-03-18,362569711,2021-03-22,1,NaT,Самовывоз,3PL 2,64865,На пути к клиенту,Кластер Екатеринбург,Кластер Северо-Запад
6594,2021-03-14,3645314759,2021-03-23,1,NaT,Самовывоз,3PL 2,9312933,На пути к клиенту,Кластер Краснодар,Кластер Казань
6595,2021-03-24,3649144391,2021-03-28,1,NaT,Курьер,Самодоставка,902912,Ожидает сборки,Кластер Екатеринбург,Кластер Екатеринбург


- Проверим есть ли заказы, в которых planned_data_order меньше, чем date_order (доставка планируется раньше самого заказа)

Такой ситуации не может быть, т. е. ошибка в данных и при подсчете метрики, эти заказы нам нужно обработать

In [233]:
df_log.query("date_order > planned_data_order")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster


- Проверим есть ли заказы, со статусом 'Доставлен', но без даты result_data_order

In [234]:
df_log.query("order_status == 'Доставлен'").isna().sum()

date_order            551
number_order            0
planned_data_order      0
timeslot_number         0
result_data_order       0
delivery_type           0
carrier_name            0
seller_ID               0
order_status            0
customer_cluster        0
warehouse_cluster       0
dtype: int64

- Проверим на наличие дубликатов в данных

Сначала просто удалим полностью идентичные строки

In [235]:
df_log = df_log.drop_duplicates()

Проверим дубликаты с timeslot_number = 1

In [236]:
df_log.query('timeslot_number == 1')\
    .groupby('number_order', as_index=False)\
    .agg({'date_order': 'count'})\
    .sort_values('date_order', ascending = False)

,number_order,date_order
922,336693909,2
756,333776216,2
2167,357469624,2
0,300612175,1
3986,3453314905,1
...,...,...
1988,354372922,1
1987,354360475,1
1986,354330926,1
1985,354309164,1


In [237]:
# список номеров заказов с дублированной первой датой заказа
misslist = df_log.query('timeslot_number == 1')\
    .groupby("number_order", as_index=False)\
    .agg({'date_order': 'count'})\
    .sort_values('date_order')\
    .query("date_order > 1")\
    .number_order

# ДФ с номерами заказа из misslist и правильной датой
misslist_m = df_log.query("number_order in @misslist")\
    .dropna()\
    .groupby('number_order', as_index=False)\
    .agg({'date_order': 'min'})\
    .rename(columns={'date_order': 'correct_date_order'})\
    .values\
    .tolist()

In [238]:
for i in range(len(misslist_m)):
    df_log['date_order'] = np.where( ((df_log.timeslot_number==1) * (df_log.number_order == misslist_m[i][0])),
                                misslist_m[i][1], 
                                df_log.date_order
                                )  
df_log['date_order'] = df_log.date_order.apply(lambda c: pd.to_datetime(c))
df_log = df_log.drop_duplicates()

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/tljh/user/lib/python3.7/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/opt/tljh/user/lib/python3.7/site-packages/ipykernel

In [239]:
df_log.query('timeslot_number == 1')\
    .groupby('number_order', as_index=False)\
    .agg({'date_order': 'count'})\
    .sort_values('date_order', ascending = False)

,number_order,date_order
2167,357469624,2
3976,3451791140,1
3985,3453214564,1
3984,3453145604,1
3983,3453037114,1
...,...,...
1986,354330926,1
1985,354309164,1
1984,354307611,1
1983,354272704,1


Проверим дубликаты с timeslot_number > 1

In [240]:
df_log.query('timeslot_number > 1').duplicated().sum()

0

# product_online

In [241]:
df_prod = product_online

**number_order** - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. так как в одном заказе может быть несколько товаров)

**product_ID** - уникальный номер товара (В столбце номер может повторяться, т.к. так как один товар может быть в заказах нескольких клиентов)

**product_price** - стоимость одной единицы товара в рублях

**amount** - количество единиц товара в заказе

In [242]:
df_prod

,number_order,product_ID,product_price,amount
0,313655114,983587801,5512.44,1
1,313655114,983587158,4070.35,2
2,313655114,983582669,3591.97,1
3,315229153,983589424,4433.23,3
4,315229153,983587827,2862.50,2
...,...,...,...,...
12228,3649144391,983583847,8146.30,2
12229,3649144391,983581209,2816.34,4
12230,365435447,983582430,2937.91,4
12231,365435447,983589903,7626.13,4


In [243]:
df_prod.dtypes

number_order       int64
product_ID         int64
product_price    float64
amount             int16
dtype: object

In [244]:
df_prod.isna().sum()

number_order     0
product_ID       0
product_price    0
amount           0
dtype: int64

# returns_online

In [245]:
df_rtrn = returns_online

In [246]:
df_rtrn 

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,1970-01-01
1,31014374140,983586554,1,2021-07-01
2,3147145173,983589156,1,2021-11-01
3,3141464669,983589362,1,2021-11-01
4,3131450699,983589729,3,2021-12-01
...,...,...,...,...
491,35714135014,983587402,2,1970-01-01
492,35714135014,983584567,3,1970-01-01
493,35714135014,983587402,2,1970-01-01
494,361501547,983581947,1,1970-01-01


In [247]:
df_rtrn .dtypes

number_order              int64
product_ID                int64
amount_return              int8
return_date      datetime64[ns]
dtype: object

In [248]:
df_rtrn .isna().sum()

number_order     0
product_ID       0
amount_return    0
return_date      0
dtype: int64

В ДФ есть строки с датой возврата '1970-01-01'

Примем, что данный заказ был возвращен в день выдачи

In [249]:
df_log

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...
6592,2021-03-14,361544120,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,64865,Доставлен,Кластер Краснодар,Кластер Северо-Запад
6593,2021-03-18,362569711,2021-03-22,1,NaT,Самовывоз,3PL 2,64865,На пути к клиенту,Кластер Екатеринбург,Кластер Северо-Запад
6594,2021-03-14,3645314759,2021-03-23,1,NaT,Самовывоз,3PL 2,9312933,На пути к клиенту,Кластер Краснодар,Кластер Казань
6595,2021-03-24,3649144391,2021-03-28,1,NaT,Курьер,Самодоставка,902912,Ожидает сборки,Кластер Екатеринбург,Кластер Екатеринбург


In [250]:
df_rtrn = df_rtrn.query("number_order in @df_log.number_order")

correct_return_date = df_log.query("number_order in @df_rtrn.number_order and result_data_order.notnull()")\
    [['number_order', 'result_data_order']]\
    .values\
    .tolist()

for i in range(len(correct_return_date)):
    df_rtrn['return_date'] = np.where( (df_rtrn.number_order==correct_return_date[i][0]),
                                correct_return_date[i][1], 
                                df_rtrn.return_date
                                )  
df_rtrn['return_date'] = df_rtrn.return_date.apply(lambda c: pd.to_datetime(c))

In [251]:
df_rtrn

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,2020-12-30
1,31014374140,983586554,1,2021-01-07
2,3147145173,983589156,1,2021-01-08
3,3141464669,983589362,1,2021-01-09
4,3131450699,983589729,3,2021-01-12
...,...,...,...,...
491,35714135014,983587402,2,2021-03-15
492,35714135014,983584567,3,2021-03-15
493,35714135014,983587402,2,2021-03-15
494,361501547,983581947,1,2021-03-15


# ЗАДАЧА 1.

Посчитать **OnTime** - Метрика своевременности доставки. (Доставили в плановую дату доставки или нет), в %

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам;

In [252]:
# Будем считать, что дата последнего заказа - это актуальная дата на момент подсчета метрики
current_day = df_log.date_order.max()
current_day

Timestamp('2021-04-26 00:00:00')

In [253]:
df1 = df_log.copy()
df1.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [254]:
df1.timeslot_number.value_counts()

1    5969
2     524
3      78
4      18
5       3
7       1
6       1
Name: timeslot_number, dtype: int64

### Датафрейм по датам

In [255]:
# Датафрейм с доставленными вовремя заказам по планируемым датам
df1a = df1.query("planned_data_order >= result_data_order")\
    .groupby('planned_data_order', as_index = False)\
    .agg({'number_order': 'count'})\
    .rename(columns = {'number_order': 'delivered_ontime_orders'})

# Датафрейм с количеством заказов по планируемым датам
df1b = df1.groupby('planned_data_order', as_index = False)\
    .agg({'number_order': 'count'})\
    .rename(columns = {'number_order': 'planned_orders'})

In [256]:
df1_total = df1a.merge(df1b, on='planned_data_order', how='right').fillna(0)

In [257]:
df1_total ['OnTime'] = df1_total.delivered_ontime_orders / df1_total.planned_orders

In [258]:
#df1_total.to_excel('df_task3_1_a.xlsx')
df1_total.head()

,planned_data_order,delivered_ontime_orders,planned_orders,OnTime
0,2021-01-01,3.0,5,0.60
1,2021-01-02,1.0,2,0.50
2,2021-01-03,1.0,2,0.50
3,2021-01-05,9.0,12,0.75
4,2021-01-06,2.0,2,1.00


In [259]:
df1_total.delivered_ontime_orders.sum()

4884.0

In [260]:
df1_total.planned_orders.sum()

6594

### Датафрейм по датам и типу доставки

In [261]:
# Датафрейм с доставленными вовремя заказам по всем параметрам
df1af = df1.query("planned_data_order >= result_data_order")\
    .groupby(['planned_data_order', 'delivery_type'], as_index = False)\
    .agg({'number_order': 'count'})\
    .rename(columns = {'number_order': 'delivered_ontime_orders'})

#Создаем уникальный индекс для merge
df1af['unique_combination'] = df1af.planned_data_order.apply(str) + df1af.delivery_type.apply(str)

# Датафрейм с количеством заказов по всем параметрам
df1bf = df1.groupby(['planned_data_order', 'delivery_type'], as_index = False)\
    .agg({'number_order': 'count'})\
    .rename(columns = {'number_order': 'planned_orders'})

#Создаем уникальный индекс для merge
df1bf['unique_combination'] = df1bf.planned_data_order.apply(str) + df1bf.delivery_type.apply(str)

df1_f = df1bf.merge(df1af[['delivered_ontime_orders', 'unique_combination']], 
                    on = 'unique_combination', how = 'left').fillna(0)

## Датафрейм по всем параметрам

In [262]:
# Датафрейм с доставленными вовремя заказам по всем параметрам
df1af = df1.query("planned_data_order >= result_data_order")\
    .groupby(['planned_data_order', 'delivery_type', 'carrier_name',
             'customer_cluster', 'warehouse_cluster', 'timeslot_number'], as_index = False)\
    .agg({'number_order': 'count'})\
    .rename(columns = {'number_order': 'delivered_ontime_orders'})

#Создаем уникальный индекс для merge
df1af['unique_combination'] = df1af.planned_data_order.apply(str) + df1af.delivery_type.apply(str) + \
    df1af.carrier_name.apply(str) + df1af.customer_cluster.apply(str) + df1af.warehouse_cluster.apply(str) + \
    df1af.timeslot_number.apply(str)

# Датафрейм с количеством заказов по всем параметрам
df1bf = df1.groupby(['planned_data_order', 'delivery_type', 'carrier_name',
             'customer_cluster', 'warehouse_cluster', 'timeslot_number'], as_index = False)\
    .agg({'number_order': 'count'})\
    .rename(columns = {'number_order': 'planned_orders'})

#Создаем уникальный индекс для merge
df1bf['unique_combination'] = df1bf.planned_data_order.apply(str) + df1bf.delivery_type.apply(str) +\
    df1bf.carrier_name.apply(str) + df1bf.customer_cluster.apply(str) + df1bf.warehouse_cluster.apply(str) + \
    df1bf.timeslot_number.apply(str)

df1_f = df1bf.merge(df1af[['delivered_ontime_orders', 'unique_combination']], 
                    on = 'unique_combination', how = 'left').fillna(0)

df1_f = df1_f.drop(columns = 'unique_combination')

In [263]:
df1_f

,planned_data_order,delivery_type,carrier_name,customer_cluster,warehouse_cluster,timeslot_number,planned_orders,delivered_ontime_orders
0,2021-01-01,Самовывоз,3PL 1,Кластер Екатеринбург,Кластер Москва,1,1,1.0
1,2021-01-01,Самовывоз,3PL 2,Кластер Екатеринбург,Кластер Москва,1,1,0.0
2,2021-01-01,Самовывоз,3PL 2,Кластер Казань,Кластер Москва,1,1,1.0
3,2021-01-01,Самовывоз,3PL 2,Кластер Краснодар,Кластер Екатеринбург,1,1,0.0
4,2021-01-01,Самовывоз,3PL 2,Кластер Краснодар,Кластер Москва,1,1,1.0
...,...,...,...,...,...,...,...,...
3877,2021-04-27,Самовывоз,3PL 1,Кластер Новосибирск,Кластер Екатеринбург,2,1,0.0
3878,2021-04-30,Курьер,Самодоставка,Кластер Казань,Кластер Московская область,1,1,0.0
3879,2021-04-30,Курьер,Самодоставка,Кластер Краснодар,Кластер Московская область,1,1,0.0
3880,2021-04-30,Курьер,Самодоставка,Кластер Москва,Кластер Московская область,1,1,0.0


**Проверка**

Должно выполнятся равенство количества заказов в ДФ с группировкой по дате и по всем параметрам

In [264]:
df1_total.planned_orders.sum() - df1_f.planned_orders.sum()

0

In [265]:
df1_total.delivered_ontime_orders.sum() - df1_f.delivered_ontime_orders.sum()

0.0

In [266]:
df1_f['customer_cluster'] = df1_f.customer_cluster.apply(lambda c: " ".join(c.split()[1:]))
df1_f['warehouse_cluster'] = df1_f.warehouse_cluster.apply(lambda c: " ".join(c.split()[1:]))

In [267]:
# Записываем ДФ в файл
df1_f.to_excel('df_task3_1.xlsx')

**ССЫЛКА НА ДАШБОРД В Tablue**

https://public.tableau.com/app/profile/ilya3868/viz/Task3_1_16629940830780/OnTime-Dashboard?publish=yes

# ЗАДАЧА 2.

Посчитать **PromisedClick2Delivery** - метрика обещанного срока доставки (от даты заказа до первой плановой даты доставки), в сутках.

 - Необходима динамика по дням, неделям, месяцам;

 - Необходимо сравнение перевозчиков;

 - Необходимо понимание уровня метрики из кластера в кластер.

In [268]:
df2 = df_log.copy()
df2.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [269]:
df2 = df2[['date_order', 'planned_data_order', 'carrier_name', 'customer_cluster', 'warehouse_cluster']]\
    .query("date_order.notnull()")
df2.head()

,date_order,planned_data_order,carrier_name,customer_cluster,warehouse_cluster
0,2021-01-05,2021-01-14,3PL 1,Кластер Краснодар,Кластер Москва
1,2021-01-03,2021-01-11,Самодоставка,Кластер Московская область,Кластер Московская область
2,2021-01-08,2021-01-16,Самодоставка,Кластер Московская область,Кластер Московская область
3,2021-01-12,2021-01-20,3PL 1,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,2021-01-26,3PL 1,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [270]:
df2['customer_cluster'] = df2.customer_cluster.apply(lambda c: " ".join(c.split()[1:]))
df2['warehouse_cluster'] = df2.warehouse_cluster.apply(lambda c: " ".join(c.split()[1:]))

In [271]:
df2

,date_order,planned_data_order,carrier_name,customer_cluster,warehouse_cluster
0,2021-01-05,2021-01-14,3PL 1,Краснодар,Москва
1,2021-01-03,2021-01-11,Самодоставка,Московская область,Московская область
2,2021-01-08,2021-01-16,Самодоставка,Московская область,Московская область
3,2021-01-12,2021-01-20,3PL 1,Санкт-Петербург,Москва
4,2021-01-16,2021-01-26,3PL 1,Санкт-Петербург,Санкт-Петербург
...,...,...,...,...,...
6592,2021-03-14,2021-03-19,3PL 2,Краснодар,Северо-Запад
6593,2021-03-18,2021-03-22,3PL 2,Екатеринбург,Северо-Запад
6594,2021-03-14,2021-03-23,3PL 2,Краснодар,Казань
6595,2021-03-24,2021-03-28,Самодоставка,Екатеринбург,Екатеринбург


In [272]:
# Записываем ДФ в файл
df2.to_excel('df_task3_2.xlsx')

**ССЫЛКА НА ДАШБОРД В Tablue**

https://public.tableau.com/app/profile/ilya3868/viz/Task3_2_16630006419330/PromisedClick2Delivery-Dashboard?publish=yes

# ЗАДАЧА 3.

Посчитать **Click2Delivery** - метрика фактического срока доставки (от даты заказа до момента вручения), в сутках.

 - Необходимо сравнение с метрикой PromisedClick2Delivery;

 - Необходимо понимание кластеров, где заказы доставляются быстрее, чем мы обещаем клиентам;

In [273]:
df3 = df_log.copy()
df3

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...
6592,2021-03-14,361544120,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,64865,Доставлен,Кластер Краснодар,Кластер Северо-Запад
6593,2021-03-18,362569711,2021-03-22,1,NaT,Самовывоз,3PL 2,64865,На пути к клиенту,Кластер Екатеринбург,Кластер Северо-Запад
6594,2021-03-14,3645314759,2021-03-23,1,NaT,Самовывоз,3PL 2,9312933,На пути к клиенту,Кластер Краснодар,Кластер Казань
6595,2021-03-24,3649144391,2021-03-28,1,NaT,Курьер,Самодоставка,902912,Ожидает сборки,Кластер Екатеринбург,Кластер Екатеринбург


In [274]:
df3 = df3[['date_order', 'planned_data_order', 'result_data_order', 'carrier_name', 'customer_cluster', 'warehouse_cluster']]\
    .query("date_order.notnull()")

In [275]:
df3['customer_cluster'] = df3.customer_cluster.apply(lambda c: " ".join(c.split()[1:]))
df3['warehouse_cluster'] = df3.warehouse_cluster.apply(lambda c: " ".join(c.split()[1:]))
df3

,date_order,planned_data_order,result_data_order,carrier_name,customer_cluster,warehouse_cluster
0,2021-01-05,2021-01-14,2021-01-11,3PL 1,Краснодар,Москва
1,2021-01-03,2021-01-11,2021-01-08,Самодоставка,Московская область,Московская область
2,2021-01-08,2021-01-16,2021-01-14,Самодоставка,Московская область,Московская область
3,2021-01-12,2021-01-20,2021-01-15,3PL 1,Санкт-Петербург,Москва
4,2021-01-16,2021-01-26,2021-01-25,3PL 1,Санкт-Петербург,Санкт-Петербург
...,...,...,...,...,...,...
6592,2021-03-14,2021-03-19,2021-03-15,3PL 2,Краснодар,Северо-Запад
6593,2021-03-18,2021-03-22,NaT,3PL 2,Екатеринбург,Северо-Запад
6594,2021-03-14,2021-03-23,NaT,3PL 2,Краснодар,Казань
6595,2021-03-24,2021-03-28,NaT,Самодоставка,Екатеринбург,Екатеринбург


In [276]:
df3 = df3.dropna()

In [277]:
# Записываем ДФ в файл
df3.to_excel('df_task3_3.xlsx')

**ССЫЛКА НА ДАШБОРД В Tablue**

https://public.tableau.com/app/profile/ilya3868/viz/Task3_3_16630657912300/Click2Delivery-Dashboard?publish=yes

# ЗАДАЧА 4.

Посчитать **GMV Accepted** - метрика совокупного объема продаж (считается по заказам, оформленным в указанный день), в рублях.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада;

In [278]:
df4_l = df_log.copy()
df4_l = df4_l[['date_order', 'number_order', 'timeslot_number',
           'delivery_type', 'carrier_name', 'customer_cluster',
           'warehouse_cluster']]
df4_l = df4_l.query("date_order.notnull()")
df4_l

,date_order,number_order,timeslot_number,delivery_type,carrier_name,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,1,Самовывоз,3PL 1,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,1,Курьер,Самодоставка,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,1,Курьер,Самодоставка,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,1,Самовывоз,3PL 1,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,1,Самовывоз,3PL 1,Кластер Санкт-Петербург,Кластер Санкт-Петербург
...,...,...,...,...,...,...,...
6592,2021-03-14,361544120,1,Самовывоз,3PL 2,Кластер Краснодар,Кластер Северо-Запад
6593,2021-03-18,362569711,1,Самовывоз,3PL 2,Кластер Екатеринбург,Кластер Северо-Запад
6594,2021-03-14,3645314759,1,Самовывоз,3PL 2,Кластер Краснодар,Кластер Казань
6595,2021-03-24,3649144391,1,Курьер,Самодоставка,Кластер Екатеринбург,Кластер Екатеринбург


In [279]:
df4_l.number_order.value_counts()

357469624      2
3514173439     1
3441343146     1
35701455514    1
353662320      1
              ..
3624411425     1
331292901      1
350477533      1
349572435      1
361144320      1
Name: number_order, Length: 5968, dtype: int64

In [280]:
df4_p = df_prod.copy()
df4_p['position_price'] = df4_p.product_price * df4_p.amount
df4_p = df4_p.groupby("number_order", as_index=False)\
    .agg({'position_price': 'sum'})
df4_p.head()

,number_order,position_price
0,300232595,13049.90
1,300612175,21582.16
2,301939627,16359.44
3,302073096,13328.32
4,302459167,8701.56


In [281]:
df4 = df4_l.merge(df4_p, on='number_order', how='left')
df4['customer_cluster'] = df4.customer_cluster.apply(lambda c: " ".join(c.split()[1:]))
df4['warehouse_cluster'] = df4.warehouse_cluster.apply(lambda c: " ".join(c.split()[1:]))
df4

,date_order,number_order,timeslot_number,delivery_type,carrier_name,customer_cluster,warehouse_cluster,position_price
0,2021-01-05,313655114,1,Самовывоз,3PL 1,Краснодар,Москва,17245.11
1,2021-01-03,315229153,1,Курьер,Самодоставка,Московская область,Московская область,19024.69
2,2021-01-08,3114106095,1,Курьер,Самодоставка,Московская область,Московская область,45515.87
3,2021-01-12,31145541407,1,Самовывоз,3PL 1,Санкт-Петербург,Москва,33376.42
4,2021-01-16,325050136,1,Самовывоз,3PL 1,Санкт-Петербург,Санкт-Петербург,25225.47
...,...,...,...,...,...,...,...,...
5964,2021-03-14,361544120,1,Самовывоз,3PL 2,Краснодар,Северо-Запад,6894.32
5965,2021-03-18,362569711,1,Самовывоз,3PL 2,Екатеринбург,Северо-Запад,28218.28
5966,2021-03-14,3645314759,1,Самовывоз,3PL 2,Краснодар,Казань,21666.50
5967,2021-03-24,3649144391,1,Курьер,Самодоставка,Екатеринбург,Екатеринбург,56010.01


In [282]:
# Записываем ДФ в файл
df4.to_excel('df_task3_4.xlsx')

**ССЫЛКА НА ДАШБОРД В Tablue**

https://public.tableau.com/app/profile/ilya3868/viz/Task3_4_16630755585930/GMV_Accepted-Dashboard?publish=yes

# ЗАДАЧА 5.

Посчитать **Return Rate** - доля возвратов (отношение количества возвратов к заказам в этот день), в %.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по кластерам склада и клиента;

In [283]:
df5_l = df4.copy()


In [284]:
df5_p = df_prod.groupby('number_order', as_index=False)\
    .agg({'amount':sum})

In [285]:
df5 = df5_l.merge(df5_p, on='number_order', how='left')\
    .rename(columns={'amount': 'goods_amount'})

In [286]:
df5 = df5.drop(columns=['timeslot_number', 'delivery_type'])

In [287]:
df5 

,date_order,number_order,carrier_name,customer_cluster,warehouse_cluster,position_price,goods_amount
0,2021-01-05,313655114,3PL 1,Краснодар,Москва,17245.11,4
1,2021-01-03,315229153,Самодоставка,Московская область,Московская область,19024.69,5
2,2021-01-08,3114106095,Самодоставка,Московская область,Московская область,45515.87,8
3,2021-01-12,31145541407,3PL 1,Санкт-Петербург,Москва,33376.42,6
4,2021-01-16,325050136,3PL 1,Санкт-Петербург,Санкт-Петербург,25225.47,11
...,...,...,...,...,...,...,...
5964,2021-03-14,361544120,3PL 2,Краснодар,Северо-Запад,6894.32,2
5965,2021-03-18,362569711,3PL 2,Екатеринбург,Северо-Запад,28218.28,6
5966,2021-03-14,3645314759,3PL 2,Краснодар,Казань,21666.50,4
5967,2021-03-24,3649144391,Самодоставка,Екатеринбург,Екатеринбург,56010.01,11


In [288]:
    # Так как в разных заказах для одинаковых товаров может быть разная цена
    # то нужно поотдельности смотреть сумму отказа для каждой комбинации (заказ, товар)
df5_p2 = df_prod.copy()
df5_p2['temp'] = df5_p2.number_order.apply(str) + ' ' + df5_p2.product_ID.apply(str)

df5_r = df_rtrn.copy()
df5_r['temp'] = df5_r.number_order.apply(str) + ' ' + df5_r.product_ID.apply(str)

df5_r = df5_r.merge(df5_p2[['temp', 'product_price']], on='temp', how='left')
df5_r = df5_r.drop(columns=['temp'])
df5_r

,number_order,product_ID,amount_return,return_date,product_price
0,3031410942,983588011,1,2020-12-30,3301.43
1,31014374140,983586554,1,2021-01-07,3457.81
2,3147145173,983589156,1,2021-01-08,6434.21
3,3141464669,983589362,1,2021-01-09,3861.63
4,3131450699,983589729,3,2021-01-12,6382.52
...,...,...,...,...,...
491,35714135014,983587402,2,2021-03-15,342.82
492,35714135014,983584567,3,2021-03-15,1610.03
493,35714135014,983587402,2,2021-03-15,342.82
494,361501547,983581947,1,2021-03-15,5461.22


In [289]:
df5_r['position_price'] = df5_r.product_price * df5_r.amount_return
df5_r

,number_order,product_ID,amount_return,return_date,product_price,position_price
0,3031410942,983588011,1,2020-12-30,3301.43,3301.43
1,31014374140,983586554,1,2021-01-07,3457.81,3457.81
2,3147145173,983589156,1,2021-01-08,6434.21,6434.21
3,3141464669,983589362,1,2021-01-09,3861.63,3861.63
4,3131450699,983589729,3,2021-01-12,6382.52,19147.56
...,...,...,...,...,...,...
491,35714135014,983587402,2,2021-03-15,342.82,685.64
492,35714135014,983584567,3,2021-03-15,1610.03,4830.09
493,35714135014,983587402,2,2021-03-15,342.82,685.64
494,361501547,983581947,1,2021-03-15,5461.22,5461.22


In [290]:
df5

,date_order,number_order,carrier_name,customer_cluster,warehouse_cluster,position_price,goods_amount
0,2021-01-05,313655114,3PL 1,Краснодар,Москва,17245.11,4
1,2021-01-03,315229153,Самодоставка,Московская область,Московская область,19024.69,5
2,2021-01-08,3114106095,Самодоставка,Московская область,Московская область,45515.87,8
3,2021-01-12,31145541407,3PL 1,Санкт-Петербург,Москва,33376.42,6
4,2021-01-16,325050136,3PL 1,Санкт-Петербург,Санкт-Петербург,25225.47,11
...,...,...,...,...,...,...,...
5964,2021-03-14,361544120,3PL 2,Краснодар,Северо-Запад,6894.32,2
5965,2021-03-18,362569711,3PL 2,Екатеринбург,Северо-Запад,28218.28,6
5966,2021-03-14,3645314759,3PL 2,Краснодар,Казань,21666.50,4
5967,2021-03-24,3649144391,Самодоставка,Екатеринбург,Екатеринбург,56010.01,11


In [291]:
df5_r = df5_r.groupby(['number_order', 'return_date'], as_index=False)\
    .agg({'position_price': 'sum', 'amount_return': 'sum'})\
    .rename(columns={'amount_return': 'goods_amount', 'return_date': 'date_order'})
df5_r

,number_order,date_order,position_price,goods_amount
0,312396246,2021-01-13,37826.64,10
1,314452924,2021-01-17,44212.16,8
2,314909631,2021-01-19,7812.37,1
3,314946464,2021-01-16,8741.31,1
4,315195519,2021-01-11,7095.66,1
...,...,...,...,...
277,339145143141,2021-02-17,5899.60,1
278,341414144254,2021-03-04,5913.58,1
279,349142014149,2021-03-01,4992.82,1
280,354140141490,2021-03-03,7611.83,1


In [292]:
df5_r['position_price'] = df5_r.position_price * (-1)
df5_r['goods_amount'] = df5_r.goods_amount * (-1)

In [293]:
df5_r

,number_order,date_order,position_price,goods_amount
0,312396246,2021-01-13,-37826.64,-10
1,314452924,2021-01-17,-44212.16,-8
2,314909631,2021-01-19,-7812.37,-1
3,314946464,2021-01-16,-8741.31,-1
4,315195519,2021-01-11,-7095.66,-1
...,...,...,...,...
277,339145143141,2021-02-17,-5899.60,-1
278,341414144254,2021-03-04,-5913.58,-1
279,349142014149,2021-03-01,-4992.82,-1
280,354140141490,2021-03-03,-7611.83,-1


In [294]:
df6_r = df5_r.copy()

In [295]:
df5_r = df5_r.merge(df5[['number_order', 'carrier_name',
                 'customer_cluster', 'warehouse_cluster']], 
            on='number_order', how='left')

df5_r = df5_r[['date_order', 'number_order', 'carrier_name', 
               'customer_cluster', 'warehouse_cluster',
               'position_price', 'goods_amount']]
df5_r

,date_order,number_order,carrier_name,customer_cluster,warehouse_cluster,position_price,goods_amount
0,2021-01-13,312396246,Самодоставка,Санкт-Петербург,Казань,-37826.64,-10
1,2021-01-17,314452924,3PL 1,Краснодар,Москва,-44212.16,-8
2,2021-01-19,314909631,3PL 1,Екатеринбург,Москва,-7812.37,-1
3,2021-01-16,314946464,3PL 3,Екатеринбург,Москва,-8741.31,-1
4,2021-01-11,315195519,Самодоставка,Московская область,Московская область,-7095.66,-1
...,...,...,...,...,...,...,...
277,2021-02-17,339145143141,Самодоставка,Екатеринбург,Екатеринбург,-5899.60,-1
278,2021-03-04,341414144254,Самодоставка,Екатеринбург,Екатеринбург,-5913.58,-1
279,2021-03-01,349142014149,3PL 2,Казань,Краснодар,-4992.82,-1
280,2021-03-03,354140141490,Самодоставка,Краснодар,Краснодар,-7611.83,-1


In [296]:
df5

,date_order,number_order,carrier_name,customer_cluster,warehouse_cluster,position_price,goods_amount
0,2021-01-05,313655114,3PL 1,Краснодар,Москва,17245.11,4
1,2021-01-03,315229153,Самодоставка,Московская область,Московская область,19024.69,5
2,2021-01-08,3114106095,Самодоставка,Московская область,Московская область,45515.87,8
3,2021-01-12,31145541407,3PL 1,Санкт-Петербург,Москва,33376.42,6
4,2021-01-16,325050136,3PL 1,Санкт-Петербург,Санкт-Петербург,25225.47,11
...,...,...,...,...,...,...,...
5964,2021-03-14,361544120,3PL 2,Краснодар,Северо-Запад,6894.32,2
5965,2021-03-18,362569711,3PL 2,Екатеринбург,Северо-Запад,28218.28,6
5966,2021-03-14,3645314759,3PL 2,Краснодар,Казань,21666.50,4
5967,2021-03-24,3649144391,Самодоставка,Екатеринбург,Екатеринбург,56010.01,11


In [297]:
df5_result = pd.concat([df5, df5_r], ignore_index=True).dropna()

In [298]:
# Записываем ДФ в файл
df5_result.to_excel('df_task3_5.xlsx')

**ССЫЛКА НА ДАШБОРД В Tablue**

https://public.tableau.com/app/profile/ilya3868/viz/Task3_5_16630932840310/Dashboard1?publish=yes

# ЗАДАЧА 6.

Посчитать **GMV D-R** - метрика, отражающая совокупный объем доставленных заказов за вычетом возвратов, в рублях.

 - Необходимо видеть изменение метрики по дням (для всех дней, которые были раньше, чем дата расчета метрики);

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада.

In [299]:
df6 = df5_result.copy()
df6

,date_order,number_order,carrier_name,customer_cluster,warehouse_cluster,position_price,goods_amount
0,2021-01-05,313655114,3PL 1,Краснодар,Москва,17245.11,4
1,2021-01-03,315229153,Самодоставка,Московская область,Московская область,19024.69,5
2,2021-01-08,3114106095,Самодоставка,Московская область,Московская область,45515.87,8
3,2021-01-12,31145541407,3PL 1,Санкт-Петербург,Москва,33376.42,6
4,2021-01-16,325050136,3PL 1,Санкт-Петербург,Санкт-Петербург,25225.47,11
...,...,...,...,...,...,...,...
6246,2021-02-17,339145143141,Самодоставка,Екатеринбург,Екатеринбург,-5899.60,-1
6247,2021-03-04,341414144254,Самодоставка,Екатеринбург,Екатеринбург,-5913.58,-1
6248,2021-03-01,349142014149,3PL 2,Казань,Краснодар,-4992.82,-1
6249,2021-03-03,354140141490,Самодоставка,Краснодар,Краснодар,-7611.83,-1


In [300]:
df_temp6 = df_log.query("number_order in @df6.number_order")\
    [['number_order', 'delivery_type']]\
    .drop_duplicates()

In [301]:
df6 = df6.merge(df_temp6, on='number_order', how='left')

In [302]:
df6

,date_order,number_order,carrier_name,customer_cluster,warehouse_cluster,position_price,goods_amount,delivery_type
0,2021-01-05,313655114,3PL 1,Краснодар,Москва,17245.11,4,Самовывоз
1,2021-01-03,315229153,Самодоставка,Московская область,Московская область,19024.69,5,Курьер
2,2021-01-08,3114106095,Самодоставка,Московская область,Московская область,45515.87,8,Курьер
3,2021-01-12,31145541407,3PL 1,Санкт-Петербург,Москва,33376.42,6,Самовывоз
4,2021-01-16,325050136,3PL 1,Санкт-Петербург,Санкт-Петербург,25225.47,11,Самовывоз
...,...,...,...,...,...,...,...,...
6246,2021-02-17,339145143141,Самодоставка,Екатеринбург,Екатеринбург,-5899.60,-1,Курьер
6247,2021-03-04,341414144254,Самодоставка,Екатеринбург,Екатеринбург,-5913.58,-1,Курьер
6248,2021-03-01,349142014149,3PL 2,Казань,Краснодар,-4992.82,-1,Самовывоз
6249,2021-03-03,354140141490,Самодоставка,Краснодар,Краснодар,-7611.83,-1,Курьер


In [303]:
# Записываем ДФ в файл
df6.to_excel('df_task3_6.xlsx')

**ССЫЛКА НА ДАШБОРД В Tablue**

https://public.tableau.com/app/profile/ilya3868/viz/Task3_6_16630953866930/Dashboard1?publish=yes